<a href="https://colab.research.google.com/github/AshakaDinith/AshakaDinith/blob/main/Code_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load your CSV
df = pd.read_csv('employee_locations.csv')

# Check for nulls or incorrect values
print(df.isnull().sum())
print(df.head())


In [ ]:
!pip install --upgrade gspread pandas gspread_dataframe


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 91.3 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.1 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.


In [ ]:
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import auth
from google.auth import default


In [ ]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)


In [ ]:
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1fWF58HwzqC3d2SPvrky9rUqXqqMoo5Qbio3fc32NomY')

In [ ]:
worksheet = sheet.worksheet('NO Route Name')
df = get_as_dataframe(worksheet)

# View first rows
df.head()

,Emp No,Emp Name,Shift,P or C,Latitude,Longitude,Location
0,1877,K Janaka vipul De silva,A,P,6.918430,80.098696,"6.91843015825977,80.098696247837"
1,1448,TS Aludeniya,A,P,6.944412,79.948731,"6.944412,79.948731"
2,I0063,RL channa Prabath,A,C,6.942952,80.138652,"6.942952,80.138652"
3,C0102,AVA Yasitha Kalhara,A,C,6.940609,79.979334,"6.940609,79.979334"
4,1086,IP Kumara,A,P,6.945947,79.954892,"6.945947,79.954892"


In [ ]:
# Show all duplicate rows based on Emp No
duplicates = df[df.duplicated(subset='Emp No', keep=False)]
print(duplicates)


    Emp No                  Emp Name Shift P or C  Latitude  Longitude  \
10    1083              HDP Prasanna     A      P  6.935481  79.984455   
12    1349             Sudath Kumara     A      P  6.953001  80.212367   
26   M2565           AMM Jayashantha     A      C  6.958624  79.966905   
27    1083              HAP Prasanna     A      P  6.949005  79.969454   
30    1354               HPDM Kumara     A      P  6.955068  79.981849   
34    1349           WMSK Weerakkodi     A      P  6.961610  79.995724   
36   E1619               EGS Chanaka     A      C  6.964723  79.988487   
40    1354              DMP Jayalath     A      P  6.966538  80.031223   
76    1429             CK Lankananda     A      P  6.952924  79.917520   
82    1592              RANS Rukshan     A      P  6.955486  79.945286   
85    1592          Nimantha Sameera     A      P  6.967319  79.945443   
90    1678        Prasad Amarathunga     G      P  6.996749  79.990286   
91    1678        Prasad Amarathunga  

In [ ]:
# Show original size
print("Original rows:", len(df))




Original rows: 120


In [ ]:
# Remove duplicate Emp No
df = df.drop_duplicates(subset='Emp No')

# Show cleaned size
print("Rows after removing duplicates:", len(df))

Rows after removing duplicates: 110


In [ ]:
from sklearn.preprocessing import StandardScaler



In [ ]:
# Create a separate DataFrame or array with just the lat/lon values
location_data = df[['Latitude', 'Longitude']]


In [ ]:
print(df[['Latitude', 'Longitude']].head())



   Latitude  Longitude
0  6.918430  80.098696
1  6.944412  79.948731
2  6.942952  80.138652
3  6.940609  79.979334
4  6.945947  79.954892


In [ ]:
# Convert to float
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

In [ ]:
df.drop(columns=['Location'], inplace=True)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_locations = scaler.fit_transform(df[['Latitude', 'Longitude']])


In [ ]:
# View first few scaled coordinates
import pandas as pd
scaled_df = pd.DataFrame(scaled_locations, columns=['Latitude_scaled', 'Longitude_scaled'])
print(scaled_df.head())


   Latitude_scaled  Longitude_scaled
0        -1.074107          1.631897
1        -0.624917         -0.660251
2        -0.650158          2.242603
3        -0.690665         -0.192498
4        -0.598379         -0.566083


In [ ]:
from sklearn.cluster import KMeans

# Define number of clusters
kmeans = KMeans(n_clusters=5, random_state=0)
df['Cluster'] = kmeans.fit_predict(scaled_locations)


In [ ]:
from sklearn.cluster import KMeans

# Choose number of clusters (e.g., total employees // 12 for van capacity)
import math
num_clusters = math.ceil(len(df) / 12)

# Run K-Means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df['Cluster'] = kmeans.fit_predict(scaled_locations)


In [ ]:
# Display a few rows
df[['Emp No', 'Latitude', 'Longitude', 'Cluster']].head(10)


,Emp No,Latitude,Longitude,Cluster
0,1877,6.918430,80.098696,3
1,1448,6.944412,79.948731,0
2,I0063,6.942952,80.138652,3
3,C0102,6.940609,79.979334,5
4,1086,6.945947,79.954892,0
5,E1409,6.950451,79.955688,0
6,1589,6.949048,79.954485,0
7,1692,6.942952,80.138652,3
8,1294,6.940609,79.979334,5
9,969,6.952198,80.212565,3


from matplotlib import pyplot as plt
_df_0['Latitude'].plot(kind='hist', bins=20, title='Latitude')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Longitude'].plot(kind='hist', bins=20, title='Longitude')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Cluster'].plot(kind='hist', bins=20, title='Cluster')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3.plot(kind='scatter', x='Latitude', y='Longitude', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_18.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_19['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_20['Latitude'].plot(kind='line', figsize=(8, 4), title='Latitude')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_21['Longitude'].plot(kind='line', figsize=(8, 4), title='Longitude')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_22['Cluster'].plot(kind='line', figsize=(8, 4), title='Cluster')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# Show how many employees are in each cluster
df.groupby('Cluster').size()


,0
Cluster,
0,24
1,11
2,3
3,11
4,3
5,22
6,27
7,1
8,4


In [ ]:
import folium
from folium.plugins import MarkerCluster

# Create a base map centered on the average location
map_center = [df['Latitude'].mean(), df['Longitude'].mean()]
cluster_map = folium.Map(location=map_center, zoom_start=9)

# Assign a color per cluster
import random
cluster_colors = {}
for cluster_id in df['Cluster'].unique():
    cluster_colors[cluster_id] = "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Add employee locations to the map
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        color=cluster_colors[row['Cluster']],
        fill=True,
        fill_opacity=0.7,
        popup=f"Emp No: {row['Emp No']}<br>Cluster: {row['Cluster']}"
    ).add_to(cluster_map)

# Save and display the map
cluster_map.save("clustered_employees_map.html")


In [ ]:
# Get cluster centers in scaled format
cluster_centers_scaled = kmeans.cluster_centers_

# Convert scaled centers back to original latitude/longitude
centroids = scaler.inverse_transform(cluster_centers_scaled)

# Add to DataFrame
pickup_points = pd.DataFrame(centroids, columns=['Latitude', 'Longitude'])
pickup_points['Cluster'] = pickup_points.index
print(pickup_points)


   Latitude  Longitude  Cluster
0  6.954227  79.938420        0
1  6.982080  80.028219        1
2  7.169023  80.124095        2
3  6.956279  80.143217        3
4  7.061510  80.008752        4
5  6.953688  79.978412        5
6  6.984972  79.977701        6
7  6.711968  79.907564        7
8  7.139067  79.945126        8
9  7.025700  79.923660        9


In [ ]:
# Add pickup points to the map
for _, row in pickup_points.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        icon=folium.Icon(color='red', icon='bus', prefix='fa'),
        popup=f"Pickup Point - Cluster {row['Cluster']}"
    ).add_to(cluster_map)

# Save updated map
cluster_map.save("pickup_points_map.html")


In [ ]:
from IPython.display import display
import folium


In [ ]:
# Center map on average location of pickup points
map_center = [pickup_points['Latitude'].mean(), pickup_points['Longitude'].mean()]
pickup_map = folium.Map(location=map_center, zoom_start=10)


In [ ]:
# Add pickup markers
for _, row in pickup_points.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Pickup Point - Cluster {row['Cluster']}",
        icon=folium.Icon(color='green', icon='bus', prefix='fa')
    ).add_to(pickup_map)


In [ ]:
# Display the map inline in Colab
pickup_map


In [ ]:
import folium
import random

# Center the map on the average employee location
map_center = [df['Latitude'].mean(), df['Longitude'].mean()]
combined_map = folium.Map(location=map_center, zoom_start=10)


In [ ]:
# Create a color for each cluster
unique_clusters = df['Cluster'].unique()
cluster_colors = {
    cluster: "#{:06x}".format(random.randint(0, 0xFFFFFF))
    for cluster in unique_clusters
}


In [ ]:
# Plot each employee
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        color=cluster_colors[row['Cluster']],
        fill=True,
        fill_opacity=0.7,
        popup=f"Emp No: {row['Emp No']}<br>Cluster: {row['Cluster']}"
    ).add_to(combined_map)


In [ ]:
# Add pickup markers (centroids)
for _, row in pickup_points.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Pickup Point - Cluster {row['Cluster']}",
        icon=folium.Icon(color='green', icon='bus', prefix='fa')
    ).add_to(combined_map)


In [ ]:
# Show interactive map inline in notebook
combined_map
